## Mediation analyais - functional - structure - behavior

#### Load RSA and SBM data

In [27]:
# specify subjects
import numpy as np
import pandas as pd
from nilearn import masking, image
import pingouin as pg
from sklearn.preprocessing import StandardScaler

In [1]:
sub_info = pd.read_csv(r'/mnt/workdir/DCM/BIDS/participants.tsv', sep='\t')
val_sub_info = sub_info.query("game1_fmri>=0.5").sort_values('Participant_ID')
val_sub_info = val_sub_info[val_sub_info['Participant_ID'] != 'sub-089']
val_sub_info = val_sub_info[val_sub_info['Participant_ID'] != 'sub-197']
val_sub = val_sub_info['Participant_ID'].to_list()

# load sbm data
sbm_metrics = pd.read_csv(r'/mnt/data/DCM/HNY/Structure/SBM/Organized/stats.thickness.aparc_AllData.csv')
# sbm_metrics = pd.read_csv(r'/mnt/data/DCM/HNY/Structure/SBM/Rawdata/基于表面的脑形态学分析_2023-03-12T18_39_14/Results/stats.volume.hippo/'
#                           r'stats.volume.hippo_AllData.csv')
sbm_metrics = sbm_metrics.drop('Group', axis=1)

# update sub_id for vbm meteric
sub_list = sbm_metrics['Subject']
sub_id_list = []
for sub in sub_list:
    sub_id = sub.split("_")[3]
    sub_id_list.append('sub-' + sub_id)

sbm_metrics['Subject'] = sub_id_list
sbm_metrics = sbm_metrics[sbm_metrics['Subject'].isin(val_sub)]
sbm_metrics = sbm_metrics.sort_values('Subject')

if sbm_metrics['Subject'].to_list() == val_sub:
    print("The subjects of two groups are aligned")
else:
    print("The subjects of two groups are not aligned")

# extract EC's data
metrics_names = sbm_metrics.columns
lec_metrics_names = [mn for mn in metrics_names if 'lh_entorhinal' in mn][0]
rec_metrics_names = [mn for mn in metrics_names if 'rh_entorhinal' in mn][0]
key_metrics = sbm_metrics[[lec_metrics_names, rec_metrics_names]]
age = val_sub_info['Age'].to_list()
game1_acc = val_sub_info['game1_acc'].to_list()
key_metrics.loc[:, 'Age'] = age
key_metrics.loc[:, 'game1_acc'] = game1_acc

# set cmp template
rsa_zmap = r'/mnt/data/DCM/result_backup/2023.5.14/Nilearn/game1/grid_rsa_corr_trials/Setall/6fold/' \
           r'{}/rsa/rsa_zscore_img_coarse_6fold.nii.gz'
sub_rsa_zmap = [rsa_zmap.format(sub) for sub in val_sub]
ec = image.load_img(r'/mnt/workdir/DCM/Docs/Mask/EC/juelich_EC_MNI152NL_prob.nii.gz')
ec = image.binarize_img(ec,5)
key_metrics['hexagon'] = np.mean(masking.apply_mask(sub_rsa_zmap, ec), axis=1)

NameError: name 'pd' is not defined

In [42]:
iv = 'Age'
m = ['hexagon']
dv = 'game1_acc'
# Instantiate the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the columns and transform
key_metrics[[iv,dv,m[0]]] = scaler.fit_transform(key_metrics[[iv,dv,m[0]]])
# Mediation analysis
mediation_results = pg.mediation_analysis(data=key_metrics, x=iv, m=m, y=dv, seed=42,n_boot=10000,return_dist=False)
print(mediation_results.round(3))

<ipython-input-42-eaa03d897da0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  key_metrics[[iv,dv,m[0]]] = scaler.fit_transform(key_metrics[[iv,dv,m[0]]])
/home/dell/Software/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/dell/Software/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pand

          path   coef     se   pval  CI[2.5%]  CI[97.5%]  sig
0  hexagon ~ X  0.143  0.070  0.043     0.005      0.281  Yes
1  Y ~ hexagon  0.299  0.068  0.000     0.165      0.432  Yes
2        Total  0.547  0.059  0.000     0.430      0.664  Yes
3       Direct  0.515  0.058  0.000     0.401      0.629  Yes
4     Indirect  0.032  0.018  0.045     0.004      0.079  Yes


In [41]:
from scipy.stats import pearsonr
pearson_corr = pearsonr(key_metrics['Age'], key_metrics['hexagon'])
print(pearson_corr)

(0.14306314326939554, 0.042759249683050116)


In [13]:
sub_info = pd.read_csv(r'/mnt/workdir/DCM/BIDS/participants.tsv', sep='\t')
val_sub_info = sub_info.query("game1_fmri>=0.5").sort_values('Participant_ID')
val_sub_info = val_sub_info[val_sub_info['Participant_ID'] != 'sub-089']
val_sub_info = val_sub_info[val_sub_info['Participant_ID'] != 'sub-197']
val_sub = val_sub_info['Participant_ID'].to_list()

# load sbm data
#sbm_metrics = pd.read_csv(r'/mnt/data/DCM/HNY/Structure/SBM/Organized/stats.volume.aparc_AllData.csv')
sbm_metrics = pd.read_csv(r'/mnt/data/DCM/HNY/Structure/SBM/Rawdata/基于表面的脑形态学分析_2023-03-12T18_39_14/Results/stats.volume.hippo/'
                          r'stats.volume.hippo_AllData.csv')
sbm_metrics = sbm_metrics.drop('Group', axis=1)

# update sub_id for vbm meteric
sub_list = sbm_metrics['Subject']
sub_id_list = []
for sub in sub_list:
    sub_id = sub.split("_")[3]
    sub_id_list.append('sub-' + sub_id)

sbm_metrics['Subject'] = sub_id_list
sbm_metrics = sbm_metrics[sbm_metrics['Subject'].isin(val_sub)]
sbm_metrics = sbm_metrics.sort_values('Subject')

if sbm_metrics['Subject'].to_list() == val_sub:
    print("The subjects of two groups are aligned")
else:
    print("The subjects of two groups are not aligned")

metrics_names = sbm_metrics.columns[1:]

The subjects of two groups are aligned


In [18]:
sign_areas = {}
for area in metrics_names:
    print(area)
    key_metrics = sbm_metrics[[area]]
    age = val_sub_info['Age'].to_list()
    game1_acc = val_sub_info['game1_acc'].to_list()
    key_metrics.loc[:, 'Age'] = age
    key_metrics.loc[:, 'game1_acc'] = game1_acc

    # mediation analysis
    iv = 'Age'
    m = [area]
    dv = 'game1_acc'
    # Instantiate the StandardScaler
    scaler = StandardScaler()

    # Fit the scaler to the columns and transform
    key_metrics[['Age','game1_acc',m[0]]] = scaler.fit_transform(key_metrics[['Age','game1_acc',m[0]]])
    # Mediation analysis
    mediation_results = pg.mediation_analysis(data=key_metrics, x=iv, m=m, y=dv, seed=42,n_boot=1000,return_dist=False)
    p = mediation_results.loc[mediation_results.path=='Indirect', 'pval'].values[0]
    if p < 0.05:
        sign_areas[area] = p
        #sign_areas.append(area)
#       print(mediation_results.round(3))

left_Hippocampal_tail


/home/dell/Software/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/dell/Software/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-18-410bfa65888b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

left_subiculum


/home/dell/Software/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/dell/Software/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-18-410bfa65888b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

left_CA1


...

/home/dell/Software/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/dell/Software/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-18-410bfa65888b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

right_HATA


/home/dell/Software/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/dell/Software/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-18-410bfa65888b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

right_Whole_hippocampus


/home/dell/Software/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/dell/Software/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-18-410bfa65888b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [24]:

for key,value in sign_areas.items():
    print(key,value)

{'left_Hippocampal_tail': 0.02, 'left_Whole_hippocampus': 0.028, 'right_Hippocampal_tail': 0.008, 'right_CA1': 0.026, 'right_molecular_layer_HP': 0.034, 'right_Whole_hippocampus': 0.01}
left_Hippocampal_tail 0.02
left_Whole_hippocampus 0.028
right_Hippocampal_tail 0.008
right_CA1 0.026
right_molecular_layer_HP 0.034
right_Whole_hippocampus 0.01
